# TensorFlow Basics

In [1]:
import tensorflow as tf
from numpy.random import RandomState

## Graph

In [2]:
a = tf.constant([1.0, 2.0], name = 'a')
b = tf.constant([2.0, 3.0], name = 'b')
result = a + b

print(a.graph is tf.get_default_graph)

False


In [3]:
# Define v in Graph g1 with initialization 0
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('v', initializer = tf.zeros_initializer()(shape = [1]))

# Define v in Graph g2 with initialization 1
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', initializer = tf.ones_initializer()(shape = [1]))

# Read v from Graph g1
with tf.Session(graph = g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse = True):
        print(sess.run(tf.get_variable('v')))

# Read v from Graph g2
with tf.Session(graph = g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse = True):
        print(sess.run(tf.get_variable('v')))

# Assign device for computation
g = tf.Graph()
with g.device('/gpu:0'):
    result = a + b

## The graph function in TensorFlow can isolate the tensor and computation

Instructions for updating:
Colocations handled automatically by placer.
[0.]
[1.]


## Tensor

In [4]:
## The tensor in TensorFlow is not implemented via matrix, but it is a ref to the result.
## We don't store numbers in tensor but the computation process to obtain the numbers.
a = tf.constant([1.0, 2.0], name = 'a')
b = tf.constant([2.0, 3.0], name = 'b')
result = tf.add(a, b, name = 'add')

print(result)

Tensor("add_2:0", shape=(2,), dtype=float32)


## Session

In [5]:
## The session is used to implement the computations that are defined before.
#with tf.Session() as sess:
#    sess.run()

sess = tf.Session()
with sess.as_default():
    print(result.eval)
    
print(sess.run(result))
print(result.eval(session = sess))
sess.close()

sess = tf.InteractiveSession()
print(result.eval())
sess.close()

<bound method Tensor.eval of <tf.Tensor 'add_2:0' shape=(2,) dtype=float32>>
[3. 5.]
[3. 5.]
[3. 5.]


## Neural Network Implementation

In [6]:
## tf.Variable is used to store and update the parameters in the neural network.

w1 = tf.Variable(tf.random_normal((2, 3), stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal((3, 1), stddev = 1, seed = 1))

x = tf.constant([[0.7, 0.9]])

# Forward pass
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()

# Initialize variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

print(sess.run(y))
sess.close()

[[3.957578]]


In [7]:
## tf.placeholder is used to provide input data

w1 = tf.Variable(tf.random_normal((2, 3), stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal((3, 1), stddev = 1, seed = 1))

x = tf.placeholder(tf.float32, shape = (1, 2), name = 'input')
z = tf.placeholder(tf.float32, shape = (3, 2), name = 'input')

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

b = tf.matmul(z, w1)
u = tf.matmul(b, w2)

sess = tf.Session()
init_go = tf.global_variables_initializer()
sess.run(init_go)

print(sess.run(y, feed_dict = {x: [[0.7, 0.9]]}))

print(sess.run(u, feed_dict = {z: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]}))

[[3.957578]]
[[3.957578 ]
 [1.1537654]
 [3.1674924]]


## A neural network case

In [8]:
# Define batch size
batch_size = 8

# Initialize weights
w1 = tf.Variable(tf.random_normal([2, 3], stddev = 1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev = 1))

# Using None is convenient to adopt different batch sizes
x = tf.placeholder(tf.float32, shape = (None, 2), name = 'x-input')
y_ = tf.placeholder(tf.float32, shape = (None, 1), name = 'y-input')

# Forward pass
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# Loss function
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)) 
                                + (1 - y_) * tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)))

# Backward pass
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)


# Generate a random dataset
rdm = RandomState(1)
dataset_size = 500
X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]


# Create a session to implement TensorFlow
with tf.Session() as sess:
    init_go = tf.global_variables_initializer()
    sess.run(init_go)
    
    # Parameters before the training
    print('w1 before training:')
    print(sess.run(w1))
    print('w2 before training:')
    print(sess.run(w2))
    print()
    
    STEPS = 5000
    
    for i in range(STEPS):
        # For every step, use sample of batch size for training
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        # Update the parameters using the batch samples
        sess.run(train_step, feed_dict = {x: X[start:end], y_: Y[start:end]})
        
        if i % 500 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict = {x: X, y_: Y})
            print('After %d training_steps, cross entropy is %g'%(i, total_cross_entropy))
    
    print()
    print('w1 after training:')
    print(sess.run(w1))
    print('w2 after training:')
    print(sess.run(w2))

w1 before training:
[[ 0.04514983  0.75495046  0.15823501]
 [ 1.270371   -0.26027292  1.6226314 ]]
w2 before training:
[[ 1.4216583 ]
 [-0.02765524]
 [ 0.16226768]]

After 0 training_steps, cross entropy is 1.06374
After 500 training_steps, cross entropy is 0.684012
After 1000 training_steps, cross entropy is 0.640312
After 1500 training_steps, cross entropy is 0.637011
After 2000 training_steps, cross entropy is 0.635366
After 2500 training_steps, cross entropy is 0.634459
After 3000 training_steps, cross entropy is 0.633978
After 3500 training_steps, cross entropy is 0.63373
After 4000 training_steps, cross entropy is 0.633607
After 4500 training_steps, cross entropy is 0.633548

w1 after training:
[[-0.12709132  1.1342261   0.04811944]
 [ 0.71344537  0.6895646   2.1491053 ]]
w2 after training:
[[ 0.83590794]
 [-0.45473632]
 [-0.456318  ]]
